In [ ]:
import glob
from collections import Counter
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
import stopwords

text = []
folder = '/srv/data/texts'
for file in os.listdir(folder):
    path = os.path.join(folder,file)
    f = open(path,'r')
    data = f.readlines()[0]
    text.append(data)
    f.close()

tokenized = []
counts = []
for t in text:
    t = stopwords.remove_stopwords(t)
    t = t.split(' ')

wordcounts = [Counter(token) for token in tokenized]
df = pd.DataFrame(counts, index=[f.split("/")[1].split(".")[0] for f in files]).fillna(0)

# normalization turned off
# sublinear term frequency scaling turned on (takes log of term frequencies and can help to de-emphasize function words like pronouns and articles.
tfidf = TfidfTransformer(norm=None, sublinear_tf=True)
results = tfidf.fit_transform(df)
   
readable_results = pd.DataFrame(results.toarray(), index=df.index, columns=df.columns)

# columns are texts
# use .head(30) to show only the top 30 terms
# sort using words with highest tfidf scores in A01932 as an example
readable_results.T.sort_values(by=["A01932"], ascending=False).head(30)
